<a href="https://colab.research.google.com/github/Yukkitsune/TBank-Analytics/blob/main/Data%20processing%20using%20SQL%20and%20Python/%D0%9F%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2%D1%8B%D0%B9_%D0%B1%D0%B8%D1%86%D0%B5%D0%BF%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тренажёр. Объединение таблиц и расчеты
**Добро пожаловать в тренажёр!**

Здесь тебе предстоит решить шесть небольших дополнительных задач, которые помогут набить руку и закрепить материал недели.

Удачи в решении задач и закреплении полученных знаний!

https://docs.google.com/spreadsheets/d/126P75QcYpIOuGpg9uVtqcb3cf70fOtqO/edit?usp=sharing&ouid=116897396846691111916&rtpof=true&sd=true

In [188]:
!gdown 126P75QcYpIOuGpg9uVtqcb3cf70fOtqO

Downloading...
From: https://drive.google.com/uc?id=126P75QcYpIOuGpg9uVtqcb3cf70fOtqO
To: /content/pandas_course_q3_2025_dataset.xlsx
100% 683k/683k [00:00<00:00, 111MB/s]


**Дано 4 таблицы:**

 1. *users* — список пользователей, дата регистрации, канал привлечения и т.д..
 2. *sessions* — список сессий пользователей.
 3. *orders* — история заказов с категориями.
 4. *payments* — список платежей.

In [189]:
import pandas as pd
import numpy as np

In [190]:
DATA_XLSX = '/content/pandas_course_q3_2025_dataset.xlsx'

users = pd.read_excel(DATA_XLSX, sheet_name='users')
sessions = pd.read_excel(DATA_XLSX, sheet_name='sessions')
orders = pd.read_excel(DATA_XLSX, sheet_name='orders')
payments = pd.read_excel(DATA_XLSX, sheet_name='payments')

sessions['session_date'] = pd.to_datetime(sessions['session_date'])
payments['payment_date'] = pd.to_datetime(payments['payment_date'])
users['signup_date'] = pd.to_datetime(users['signup_date'])
orders['order_date'] = pd.to_datetime(orders['order_date'])

### Задание 1

Посчитать ARPU за июль и август 2025.
ARPU = (суммарная net_revenue за месяц) / (MAU за месяц, где MAU — число уникальных user_id с ≥1 сессией в месяце).
Подсказка:



из sessions собрать MAU по month,

из payments — выручку по month,

merge по месяцу и посчитать ARPU.

In [191]:
sessions['month'] = sessions['session_date'].dt.to_period('M').astype(str)
payments['month'] = payments['payment_date'].dt.to_period('M').astype(str)

In [192]:
mau = sessions.groupby('month')['user_id'].nunique().rename('mau').reset_index()
rev = payments.groupby('month')['net_revenue'].sum().rename('revenue').reset_index()
arpu = mau.merge(rev, how='left', on='month')
arpu['arpu'] = arpu['revenue'] / arpu['mau']
arpu[arpu['month'].isin(['2025-07','2025-08'])]

,month,mau,revenue,arpu
1,2025-07,1383,4008.39,2.89833
2,2025-08,2273,6911.76,3.04081


### Задание 2

Канальная аналитика за Q3’2025 (июль–сентябрь): Вывести краткую таблицу с метриками по аквизиции приложения.

Метрики по acquisition_channel:
* revenue_q3 — суммарная net_revenue (из payments за 2025-07–2025-09).
* purchasers_q3 — число покупателей (у кого в orders есть ≥1 заказ в периоде).
* arppu_q3 — revenue_q3 / purchasers_q3.
* signups_q3 — число пользователей, зарегистрированных в Q3 (из users по дате).
* conversion_to_purchase_% — purchasers_q3 / signups_q3 * 100.
* avg_orders_per_purchaser — среднее число заказов на покупателя.

Подсказка по пайплайну:
1. Сначала сделайте фильтры по датам для payments, orders, users.
2. Посчитайте все составляющие через groupby().agg(...).
3. Склейте через merge или аккуратный pd.concat по каналу.

Пример итоговой таблицы

| acquisition_channel | revenue_q3 | purchasers_q3 | avg_orders_per_purchaser | signups_q3 | conversion_to_purchase_% | arppu_q3   |
|---------------------|-------------|----------------|---------------------------|-------------|-------------------------|------------|
| Organic             | 14046.0     | 309            | 1.145631                  | 1178        | 26.2309                 | 45.456311  |

In [270]:
### Твой код здесь
q3_start = pd.Timestamp('2025-07-01')
q3_end = pd.Timestamp('2025-09-30')

# Найдем revenue по каналу
payments_q3 = payments.loc[payments['payment_date'].between(q3_start, q3_end)]
revenue_q3_users = (payments_q3
                    .groupby('user_id')['net_revenue']
                    .sum()
                    .reset_index()
                    .merge(users[['user_id','acquisition_channel']], how='left', on='user_id')
                    )
revenue_by_channel = (revenue_q3_users
                      .groupby('acquisition_channel', as_index=False)
                      .agg(revenue_q3=('net_revenue', 'sum'))
                      )

# Найдем purchasers по каналу
orders_q3 = orders.loc[orders['order_date'].between(q3_start, q3_end)]
purchasers_q3 = (orders_q3.groupby('user_id')['order_id']
                 .nunique()
                 .reset_index(name='orders_count')
                 .assign(is_purchaser=1)
                 .merge(users[['user_id','acquisition_channel']], how='left', on='user_id')
                 )
purchasers_by_channel = (purchasers_q3
                         .groupby("acquisition_channel")["is_purchaser"]
                         .sum()
                         .rename("purchasers_q3")
                         .reset_index())

# Найдем signups по каналу
users_q3 = users.loc[users['signup_date'].between(q3_start, q3_end)]
signups_by_channel = users_q3.groupby('acquisition_channel')['user_id'].nunique().rename('signups_q3').reset_index()

# Среднее число заказов на покупателя по каналу
avg_orders = (purchasers_q3
              .groupby('acquisition_channel')['orders_count']
              .mean()
              .rename('avg_orders_per_purchaser')
              )

# arppu_q3 — revenue_q3 / purchasers_q3.
arppu_q3 = revenue_by_channel.merge(purchasers_by_channel, how='left', on='acquisition_channel')
arppu_q3['arppu_q3'] = arppu_q3['revenue_q3'] / arppu_q3['purchasers_q3']

# conversion_to_purchase_% — purchasers_q3 / signups_q3 * 100.
conversion_to_purchase_df = purchasers_by_channel.merge(signups_by_channel, how='left',on='acquisition_channel')
conversion_to_purchase_df['conversion_to_purchase_%'] = (purchasers_by_channel['purchasers_q3'] / signups_by_channel['signups_q3'] * 100)

# Склеим результаты через merge

acquisition_channel_analytics = (revenue_by_channel
                                 .merge(purchasers_by_channel, how='left',on='acquisition_channel')
                                 .merge(avg_orders, how='left',on='acquisition_channel')
                                 .merge(signups_by_channel, how='left', on='acquisition_channel')
                                 .merge(conversion_to_purchase_df[['acquisition_channel','conversion_to_purchase_%']], how='left', on='acquisition_channel')
                                 .merge(arppu_q3[['acquisition_channel','arppu_q3']], how='left', on='acquisition_channel')
                                 )
acquisition_channel_analytics = acquisition_channel_analytics.sort_values(by='revenue_q3', ascending=False)

In [271]:
acquisition_channel_analytics

,acquisition_channel,revenue_q3,purchasers_q3,avg_orders_per_purchaser,signups_q3,conversion_to_purchase_%,arppu_q3
1,Organic,14046.00,309,1.145631,1178,26.230900,45.456311
0,Ads,7308.65,174,1.149425,679,25.625920,42.003736
3,Referral,4991.47,113,1.221239,322,35.093168,44.172301
4,Social,3541.75,84,1.119048,401,20.947631,42.163690
2,Partner,2060.57,53,1.188679,173,30.635838,38.878679


### Задание 3

Когорты по месяцу регистрации. Накопительная средняя выручка на пользователя по месяцам с момента регистрации (0…3).
Шаги:

* orders + payments → склейка по order_id (чтобы получить payment_date и net_revenue).

* merge с users ради signup_date и когорты cohort = signup_``[date.to](http://date.to)``_period('M').

* Рассчитать months_since_signup для платежа: разница в месяцах между payment_date и signup_date.

* По каждому пользователю внутри когорты — накопительная сумма net_revenue по возрастанию months_since_signup (groupby(['cohort','user_id']).cumsum()).

* Затем усреднить по пользователям в когорте для каждого months_since_signup.

Пример итоговой таблицы

| cohort     | months_since_signup | avg_cum_revenue_per_user |
|------------|---------------------|---------------------------|
| 2025-06    | 0                   | 34.909048                 |

In [312]:
### Твой код здесь
orders_pay = (orders
              .merge(payments[['order_id','payment_date','net_revenue']], how='left',on='order_id')
              .merge(users[['user_id','signup_date']], how='left',on='user_id'))
orders_pay['payment_date'] = pd.to_datetime(orders_pay['payment_date'])
orders_pay['order_date'] = pd.to_datetime(orders_pay['order_date'])
orders_pay['signup_date'] = pd.to_datetime(orders_pay['signup_date'])
orders_pay['cohort'] = orders_pay['signup_date'].dt.to_period('M').astype(str)

def month_diff(a,b): return (a.year - b.year) * 12 + (a.month - b.month)

orders_pay['months_since_signup'] = orders_pay.apply(lambda x: month_diff(x['payment_date'], x['signup_date']), axis=1)
ltv_user = (orders_pay
            .groupby(['user_id', 'cohort', 'months_since_signup'])['net_revenue']
            .sum()
            .reset_index()
            .sort_values(by=['cohort','user_id','months_since_signup'])
            )
ltv_user['cum_rev_user'] = (ltv_user
                            .groupby(['cohort','user_id'])['net_revenue']
                            .cumsum()
                            )
cohorts_by_month = (ltv_user
                    .groupby(['cohort','months_since_signup'], as_index=False)
                    .agg(avg_cum_revenue_per_user=('cum_rev_user','mean')))
cohorts_by_month = cohorts_by_month[cohorts_by_month['months_since_signup'].between(0,3)]

In [313]:
cohorts_by_month

,cohort,months_since_signup,avg_cum_revenue_per_user
0,2025-06,0,34.909048
1,2025-06,1,43.161538
2,2025-06,2,38.858919
3,2025-06,3,38.776552
4,2025-07,0,44.410250
5,2025-07,1,42.408333
6,2025-07,2,45.626338
7,2025-08,0,35.442429
8,2025-08,1,49.081274
9,2025-09,0,41.381065


### Задание 4& Cегментный LTV

LTV90 (средний доход за первые 90 дней после регистрации) по паре сегментов acquisition_channel × device.
Шаги:

* На базе датасета из Задания 3 посчитайте days_since_signup = payment_date - signup_date.
* Отфильтруйте платежи с 0 ≤ days_since_signup ≤ 90.
* Для каждого пользователя посчитайте sum(net_revenue) → это LTV90 пользователя.
* Сгруппируйте по acquisition_channel и device и усредните (средний LTV90).

Пример выходной таблицы

| acquisition_channel | device | ltv90      |
|---------------------|--------|------------|
| Social              | Web    | 63.524615  |


In [327]:
### Твой код здесь
orders_pay2 = orders_pay.merge(users[['user_id','acquisition_channel','device']], how='left', on='user_id')
orders_pay2['days_since_signup'] = (orders_pay2['payment_date'] - orders_pay2['signup_date']).dt.days
ltv90_user = (orders_pay2
              .loc[orders_pay2['days_since_signup'].between(0,90)]
              .groupby(['acquisition_channel','device','user_id'], as_index=False)
              .agg(net_revenue=('net_revenue', 'sum')))
ltv90_mean = (ltv90_user
              .groupby(['acquisition_channel','device'], as_index=False)
              .agg(ltv90=('net_revenue','mean'))
              .sort_values(by='ltv90',ascending=False))

In [329]:
ltv90_mean

,acquisition_channel,device,ltv90
13,Social,Web,63.524615
1,Ads,Web,51.402333
4,Organic,Web,50.530690
10,Referral,Web,50.470833
3,Organic,Android,45.646727
7,Partner,Web,44.815714
5,Organic,iOS,44.388209
11,Referral,iOS,43.491333
9,Referral,Android,43.433488
0,Ads,Android,40.949706


### Задание 5. Со звездочкой. Возможно забрать на дом)

Product/Analytics thinking на основе данных:

1. Найдите 2–3 недооценённых сегмента, где (а) конверсия в первую покупку ниже, чем по базе, но (б) LTV90/ARPPU потенциально выше среднего (например, Referral на iOS и т.п.).

2. Предложите эксперименты/улучшения и метрики успеха:
примеры: персональные скидки для первого заказа, бандлы для Electronics, упрощение флоу на Web, пост-покупочные cross-sell для Digital, и т.д.
укажите, какие срезы и кумулятивные метрики будете мониторить (ARPU/ARPPU, avg_orders_per_purchaser, CR к 1-й/2-й покупке, LTV30/60/90, долю возвратов).

Это задание без единственно верного ответа: важно обоснование, прозрачная методика и корректное использование groupby/agg/merge.


**Подсказки для возможных направлений ответа (не единственные)**:

* Сегмент Ads может иметь невысокую конверсию → гипотеза про welcome-оферы/тест промо-механик.
* Web часто «холоднее», чем iOS/Android → гипотеза про UX friction и прогрев через контент/онбординг.
* Apparel в августе мог «подрасти» → гипотеза про сезонные кампании и перенос лучших практик на другие месяцы/категории.
* Анализ возвратов (refunded) по категориям/каналам — кандидаты на улучшение качества/описаний/ожиданий.

In [370]:
### Твой код здесь

# Conversion
users_q3 = (users.loc[users['signup_date'].between(pd.Timestamp('2025-07-01'), pd.Timestamp('2025-09-30'))]).copy()
users_q3_seg = users_q3[['user_id','acquisition_channel','device']].copy()
first_orders = (orders
                .sort_values(by=['user_id','order_date'])
                .query('order_rank == 1')[['user_id','order_date']]
                )
users_q3_with_first = users_q3.merge(first_orders, how='left',on='user_id')
users_q3_with_first['is_buyer'] = users_q3_with_first['order_date'].notna().astype(int)
conversion_first = (
    users_q3_with_first
    .groupby(['acquisition_channel','device'], as_index=False)
    .agg(signups=('user_id','nunique'),
         buyers=('is_buyer','sum'))
)
conversion_first['conversion_%'] = (
    conversion_first['buyers'] / conversion_first['signups'] * 100
)
conversion_first = conversion_first.sort_values(by='conversion_%', ascending=False)

# LTV90
orders_pay2 = orders_pay.merge(users[['user_id','acquisition_channel','device']], how='left', on='user_id')
orders_pay2['days_since_signup'] = (orders_pay2['payment_date'] - orders_pay2['signup_date']).dt.days
ltv90_user = (orders_pay2
              .loc[orders_pay2['days_since_signup'].between(0,90)]
              .groupby(['acquisition_channel','device','user_id'], as_index=False)
              .agg(net_revenue=('net_revenue', 'sum')))
ltv90_seg = (ltv90_user
              .groupby(['acquisition_channel','device'], as_index=False)
              .agg(ltv90=('net_revenue','mean'))
              .sort_values(by='ltv90',ascending=False))

# Недооцененные сегменты
conv_ltv = conversion_first.merge(ltv90_seg, how='left', on=['acquisition_channel','device'])
base_conv = conv_ltv['conversion_%'].median()
base_ltv90 = ltv90_seg['ltv90'].median()
candidates = (
    conv_ltv.assign(is_low_conv=lambda x: x['conversion_%'] < base_conv,
                    is_high_ltv=lambda x: x['ltv90'] > base_ltv90)
    .query('is_low_conv and is_high_ltv')
    .sort_values(['ltv90','conversion_%'], ascending=[False, True])
)

In [371]:
candidates

,acquisition_channel,device,signups,buyers,conversion_%,ltv90,is_low_conv,is_high_ltv
14,Social,Web,75,11,14.666667,63.524615,True,True
11,Ads,Web,149,26,17.449664,51.402333,True,True
8,Organic,Web,240,49,20.416667,50.530690,True,True
9,Organic,Android,459,92,20.043573,45.646727,True,True
13,Partner,Web,33,5,15.151515,44.815714,True,True
